In [1]:
import torch.nn            as nn
import numpy               as np
import matplotlib.pyplot   as plt
import torch.nn.functional as F
import networkx            as nx
import itertools
import torch

from os                   import path
from torch_geometric.data import Data
from pymatgen.core        import Structure
from torch.nn             import Linear

import sys
sys.path.append('../')
import MP.MP_library as MPL

# Graph embedding

In [2]:
distance_threshold = 6
y = 1
cell, composition, concentration, positions = MPL.information_from_VASPfile('.',
                                                                            'POSCAR')

nodes, edges, attributes = MPL.graph_POSCAR_encoding(None,
                                                    cell,
                                                    composition,
                                                    concentration,
                                                    positions,
                                                    distance_threshold=distance_threshold
                                                   )

#attributes = attributes * 100
temp = Data(x=nodes,
            edge_index=edges.t().contiguous(),
            edge_attr=attributes.flatten(),
            y=torch.tensor([y], dtype=torch.float)
           )
temp

Data(x=[71, 4], edge_index=[2, 931], edge_attr=[931], y=[1])

In [3]:
def allocate_4n(P1, P2, P3, P4, r1, r2, r3, r4):
    # Calculate vector representations of the reference points
    p1 = np.array([0, 0, 0])
    p2 = np.array([P2[0] - P1[0], P2[1] - P1[1], P2[2] - P1[2]])
    p3 = np.array([P3[0] - P1[0], P3[1] - P1[1], P3[2] - P1[2]])

    # Calculate vectors representing the directions from the first reference point
    # to the second and third reference points
    v1 = p2 - p1
    v2 = p3 - p1

    # Calculate unit vectors in the Xn, Yn, and Zn directions
    Xn = v1 / np.linalg.norm(v1)
    tmp = np.cross(v1, v2)
    Zn = tmp / np.linalg.norm(tmp)
    Yn = np.cross(Xn, Zn)

    # Calculate dot products and distances
    i = np.dot(Xn, v2)
    d = np.dot(Xn, v1)
    j = np.dot(Yn, v2)

    # Calculate coordinates of the unknown point in two possible solutions
    X = (np.power(r1, 2) - np.power(r2, 2) + np.power(d, 2)) / (2 * d)
    Y = (np.power(r1, 2) - np.power(r3, 2) + np.power(i, 2) + np.power(j, 2)) / (2 * j) - X * i / j    
    Z1 = np.sqrt(max(0, np.power(r1, 2) - np.power(X, 2) - np.power(Y, 2)))
    Z2 = -Z1

    # Calculate the coordinates of the unknown point for both solutions
    K1 = P1 + X * Xn + Y * Yn + Z1 * Zn
    K2 = P1 + X * Xn + Y * Yn + Z2 * Zn
    
    fourth_threshold = 1e-4
    
    if np.abs(np.linalg.norm(K1 - P4) - r4) < fourth_threshold:
        return K1
    elif np.abs(np.linalg.norm(K2 - P4) - r4) < fourth_threshold:
        return K2
    return None


def allocate_atom_3n(P1, P2, P3, r1, r2, r3):
    # Calculate vector representations of the reference points
    p1 = np.array([0, 0, 0])
    p2 = np.array([P2[0] - P1[0], P2[1] - P1[1], P2[2] - P1[2]])
    p3 = np.array([P3[0] - P1[0], P3[1] - P1[1], P3[2] - P1[2]])

    # Calculate vectors representing the directions from the first reference point
    # to the second and third reference points
    v1 = p2 - p1
    v2 = p3 - p1

    # Calculate unit vectors in the Xn, Yn, and Zn directions
    Xn = v1 / np.linalg.norm(v1)
    tmp = np.cross(v1, v2)
    Zn = tmp / np.linalg.norm(tmp)
    Yn = np.cross(Xn, Zn)

    # Calculate dot products and distances
    i = np.dot(Xn, v2)
    d = np.dot(Xn, v1)
    j = np.dot(Yn, v2)

    # Calculate coordinates of the unknown point in two possible solutions
    X = (np.power(r1, 2) - np.power(r2, 2) + np.power(d, 2)) / (2 * d)
    Y = (np.power(r1, 2) - np.power(r3, 2) + np.power(i, 2) + np.power(j, 2)) / (2 * j) - X * i / j
    Z1_square = np.power(r1, 2) - np.power(X, 2) - np.power(Y, 2)
    
    if Z1_square < -1e-4:
        return None
    
    Z1 = np.sqrt(max(0, Z1_square))
    Z2 = -Z1

    # Calculate the coordinates of the unknown point for both solutions
    K1 = P1 + X * Xn + Y * Yn + Z1 * Zn
    K2 = P1 + X * Xn + Y * Yn + Z2 * Zn
    return K1, K2

In [95]:
def get_n_connected(idx_0, cartesian_positions, edge_indexes, edge_attributes):
    n_connected = 0
    idx_connected = []
    for idx_t in list(cartesian_positions.keys()):
        if get_distance_attribute(idx_0, idx_t, edge_indexes, edge_attributes) is not None:
            n_connected += 1
            idx_connected.append(idx_t)
    
    return n_connected, idx_connected


def are_4_points_collinear(point1, point2, point3, point4, ratio_threshold=1e-3):
    # Calculate vectors formed by the points
    vector1 = [(point2[i] - point1[i]) for i in range(3)]
    vector2 = [(point3[i] - point2[i]) for i in range(3)]
    vector3 = [(point4[i] - point3[i]) for i in range(3)]

    # Calculate cross products
    cross_product1 = [
        vector1[1] * vector2[2] - vector1[2] * vector2[1],
        vector1[2] * vector2[0] - vector1[0] * vector2[2],
        vector1[0] * vector2[1] - vector1[1] * vector2[0]
    ]

    cross_product2 = [
        vector2[1] * vector3[2] - vector2[2] * vector3[1],
        vector2[2] * vector3[0] - vector2[0] * vector3[2],
        vector2[0] * vector3[1] - vector2[1] * vector3[0]
    ]

    # Check if the cross products are proportional
    for i in range(3):
        if cross_product2[i] != 0:
            ratio = np.abs(cross_product1[i] / cross_product2[i])
            if ratio > ratio_threshold:
                return False
    return True


def are_3_points_collinear(point1, point2, point3, ratio_threshold=1e-3):
    # Calculate vectors formed by the points
    vector1 = [(point2[i] - point1[i]) for i in range(3)]
    vector2 = [(point3[i] - point2[i]) for i in range(3)]

    # Calculate cross product
    cross_product = [
        vector1[1] * vector2[2] - vector1[2] * vector2[1],
        vector1[2] * vector2[0] - vector1[0] * vector2[2],
        vector1[0] * vector2[1] - vector1[1] * vector2[0]
    ]

    # Check if the cross product is zero
    return all(component < ratio_threshold for component in cross_product)


def get_distance_attribute(index0, index1, edge_indexes, edge_attributes):
    # Create a mask to find matching edges
    mask_direct  = (edge_indexes[0] == index0) & (edge_indexes[1] == index1)
    mask_reverse = (edge_indexes[0] == index1) & (edge_indexes[1] == index0)
    
    # Check if any edge satisfies the conditions
    matching_edge_indices = np.where(mask_direct | mask_reverse)[0]
    
    if len(matching_edge_indices) == 0:
        return None  # The pair is not linked
    
    # Get the distance attribute from the first matching edge
    distance_attribute = edge_attributes[matching_edge_indices[0]]
    
    return distance_attribute


def find_initial_basis(total_particles, edge_indexes, edge_attributes):
    for idx_0 in range(total_particles):
        for idx_1 in np.arange(idx_0+1, total_particles):
            for idx_2 in np.arange(idx_1+1, total_particles):
                cdn_01 = (get_distance_attribute(idx_0, idx_1, edge_indexes, edge_attributes) is not None)
                cdn_12 = (get_distance_attribute(idx_1, idx_2, edge_indexes, edge_attributes) is not None)
                cdn_02 = (get_distance_attribute(idx_0, idx_2, edge_indexes, edge_attributes) is not None)
                if cdn_01 and cdn_12 and cdn_02:
                    # Get necessary distances
                    d_01 = get_distance_attribute(idx_0, idx_1, edge_indexes, edge_attributes)
                    d_02 = get_distance_attribute(idx_0, idx_2, edge_indexes, edge_attributes)
                    d_12 = get_distance_attribute(idx_1, idx_2, edge_indexes, edge_attributes)

                    # Reference the first three atoms
                    x2 = (np.power(d_01, 2) + np.power(d_02, 2) - np.power(d_12, 2)) / (2 * d_01)
                    y2 = np.sqrt(np.power(d_02, 2) - np.power(x2, 2))

                    r0 = np.array([0,    0,  0])
                    r1 = np.array([d_01, 0,  0])
                    r2 = np.array([x2,   y2, 0])

                    # Check non-collinearity
                    if not are_3_points_collinear(r0, r1, r2):
                        return idx_0, idx_1, idx_2, r0, r1, r2


def find_valid_reference(n_connected, idx_connected, edge_indexes, edge_attributes, idx):
    if n_connected >= 4:
        for i in range(n_connected):
            for j in np.arange(i+1, n_connected):
                for k in np.arange(j+1, n_connected):
                    for l in np.arange(k+1, n_connected):
                        idx_0 = idx_connected[i]
                        idx_1 = idx_connected[j]
                        idx_2 = idx_connected[k]
                        idx_3 = idx_connected[l]

                        r0 = cartesian_positions[idx_0]
                        r1 = cartesian_positions[idx_1]
                        r2 = cartesian_positions[idx_2]
                        r3 = cartesian_positions[idx_3]

                        # Get necessary distances
                        d_0n = get_distance_attribute(idx_0, idx, edge_indexes, edge_attributes)
                        d_1n = get_distance_attribute(idx_1, idx, edge_indexes, edge_attributes)
                        d_2n = get_distance_attribute(idx_2, idx, edge_indexes, edge_attributes)
                        d_3n = get_distance_attribute(idx_3, idx, edge_indexes, edge_attributes)

                        temp_position = allocate_4n(r0, r1, r2, r3, d_0n, d_1n, d_2n, d_3n)

                        if temp_position is not None:
                            return [temp_position]
    elif n_connected == 3:
        idx_0 = idx_connected[0]
        idx_1 = idx_connected[1]
        idx_2 = idx_connected[2]

        r0 = cartesian_positions[idx_0]
        r1 = cartesian_positions[idx_1]
        r2 = cartesian_positions[idx_2]

        # Get necessary distances
        d_0n = get_distance_attribute(idx_0, idx, edge_indexes, edge_attributes)
        d_1n = get_distance_attribute(idx_1, idx, edge_indexes, edge_attributes)
        d_2n = get_distance_attribute(idx_2, idx, edge_indexes, edge_attributes)

        temp_position = allocate_atom_3n(r0, r1, r2, d_0n, d_1n, d_2n)

        if temp_position is not None:
            return temp_position
    else:
        sys.exit(f'Error: only {n_connected} particles, not enough.')

### Fix 1, 2, 3

In [103]:
# Extract indexes and attributes from the graph
edge_indexes    = edges.T.detach().cpu().numpy().copy()
edge_attributes = attributes.detach().cpu().numpy().copy().flatten()

# Define the number of atoms in the graph
total_particles = len(nodes)

# Select three initial particles which are interconnected (non-collinearity is checked)
idx_0, idx_1, idx_2, r0, r1, r2 = find_initial_basis(total_particles, edge_indexes, edge_attributes)

# Fix three particles at the beginning
cartesian_positions = {
    idx_0: r0,
    idx_1: r1,
    idx_2: r2
}

# Generate full indexes list
connected_idxs = np.arange(total_particles).tolist()

# Remove already fixed indexes
connected_idxs.remove(idx_0)
connected_idxs.remove(idx_1)
connected_idxs.remove(idx_2)

cartesian_positions

{0: array([0, 0, 0]),
 2: array([4.1835289, 0.       , 0.       ]),
 4: array([2.09514309, 3.65975825, 0.        ])}

### Impose 4

In [104]:
# Impose a fourth particle
cartesian_positions_list = cartesian_positions.values()
for idx_3 in connected_idxs:
    n_connected, _ = get_n_connected(idx_3, cartesian_positions, edge_indexes, edge_attributes)
    if n_connected >= 3:
        # Get necessary distances
        d_0n = get_distance_attribute(idx_0, idx_3, edge_indexes, edge_attributes)
        d_1n = get_distance_attribute(idx_1, idx_3, edge_indexes, edge_attributes)
        d_2n = get_distance_attribute(idx_2, idx_3, edge_indexes, edge_attributes)
        
        if (d_0n is not None) and (d_1n is not None) and (d_2n is not None):
            temp_position = allocate_atom_3n(r0, r1, r2, d_0n, d_1n, d_2n)[0]
            
            # Check non-collinearity
            if not are_4_points_collinear(r0, r1, r2, temp_position):
                # Generate temporal dictionary with the cartesian coordinates
                temp_dict = {
                    idx_3: temp_position
                }

                # Update general dictionary with cartesian coordinates
                cartesian_positions.update(temp_dict)
                connected_idxs.remove(idx_3)
                break

cartesian_positions

{0: array([0, 0, 0]),
 2: array([4.1835289, 0.       , 0.       ]),
 4: array([2.09514309, 3.65975825, 0.        ]),
 30: array([-0.10061269, -0.01397474,  3.84558059])}

### Main loop

In [105]:
def are_correctly_unconnected(temp_position, idx_unconnected, cartesian_positions):
    dist_th = 0.5
    for idx in list(cartesian_positions.keys()):
        if idx in idx_unconnected:
            aux = np.linalg.norm(cartesian_positions[idx] - temp_position)
            print(aux)
            if aux < (distance_threshold + dist_th):
                # Invalid position
                return False
    return True

In [110]:
for idx in list(cartesian_positions.keys()):
        if idx in idx_unconnected:
            print('a')

In [111]:
cartesian_positions

{0: array([0, 0, 0]),
 2: array([4.1835289, 0.       , 0.       ]),
 4: array([2.09514309, 3.65975825, 0.        ]),
 30: array([-0.10061269, -0.01397474,  3.84558059])}

In [109]:
idx in idx_unconnected

True

In [108]:
idx_unconnected

[1,
 3,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70]

In [106]:
while len(connected_idxs):  # Goes until all particles have been studied
    # Using a first-one-first-out approach
    idx = connected_idxs[0]

    n_connected, idx_connected = get_n_connected(idx, cartesian_positions, edge_indexes, edge_attributes)
    idx_unconnected = list(set(connected_idxs) - set(idx_connected))
    
    print()
    print(idx)
    print(connected_idxs)

    # Remove idx from connected_dict
    connected_idxs.remove(idx)
    
    # Set idx in cartesian_positions or else add it to connected_idxs
    if n_connected >= 3:
        # Extract the cartesian coordinates of idx
        temp_position = find_valid_reference(n_connected, idx_connected, edge_indexes, edge_attributes, idx)
        
        if temp_position is None:
            # connected_idxs is updated adding idx where it belongs to
            connected_idxs.append(idx)
            break

        for pos in temp_position:
            # Check unconnections
            if are_correctly_unconnected(pos, idx_unconnected, cartesian_positions):
                # Generate temporal dictionary with the cartesian coordinates
                temp_dict = {
                    idx: pos
                }

                # Update general dictionary with cartesian coordinates
                cartesian_positions.update(temp_dict)# Check unconnections
                break
    else:
        # connected_idxs is updated adding idx where it belongs to
        connected_idxs.append(idx)


1
[1, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70]

3
[3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 1]

5
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 1, 3]

6
[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 5

In [94]:
cartesian_positions

{0: array([0, 0, 0]),
 2: array([4.1835289, 0.       , 0.       ]),
 4: array([2.09514309, 3.65975825, 0.        ]),
 30: array([-0.10061269, -0.01397474,  3.84558059])}

In [ ]:
for i in range(total_particles):
    print(cartesian_positions[i])